# Imports

In [3]:
import xmltodict
import re
from tqdm.notebook import tqdm
import subprocess

ModuleNotFoundError: No module named 'xmltodict'

# XML Parser

In [ ]:
xml_file = 'config_tigerLake.xml'
with open(xml_file, 'r') as file:
    xml_data = file.read()

xml_dict = xmltodict.parse(xml_data)

# Stat File Parser

In [ ]:
def extract_stats(stat_file):
    # Read the configuration file
    with open(stat_file, "r") as file:
        config_data = file.read()

    # Define regex patterns for each metric
    l1d_hit_rate_pattern = r"L1\[0\] Hit-Rate\s*=\s*([\d.]+)"
    l1i_hit_rate_pattern = r"I1\[0\] Hit-Rate\s*=\s*([\d.]+)"
    l2_hit_rate_pattern = r"L2\[0\] Hit-Rate\s*=\s*([\d.]+)"
    l3_hit_rate_pattern = r"L3\[0\] Hit-Rate\s*=\s*([\d.]+)"
    l1i_tlb_hit_rate_pattern = r"iTLB\[0\] Hit-Rate\s*=\s*([\d.]+)"
    l1d_tlb_hit_rate_pattern = r"dTLB\[0\] Hit-Rate\s*=\s*([\d.]+)"
    ipc_pattern = r"IPC\s*=\s*([\d.]+)\s+in terms of micro-ops"
    branch_prediction_accuracy_pattern = r"branch predictor accuracy\s*=\s*([\d.]+)"
    total_execution_time_pattern = r"Total Execution Time\s*=\s*([\d\s:]+)"
    micro_op_cache_hit_rate_pattern = r"micro-op cache hit rate\s*=\s*([\d.]+)"
    target_predictor_accuracy_pattern = r"target predictor accuracy\s*=\s*([\d.]+)"
    predicate_predictor_accuracy_pattern = r"predicate predictor accuracy\s*=\s*([\d.]+)"
    
    core_energy_pattern = r"^coreEnergy.total*"
    shared_cache_energy_pattern = r"sharedCacheEnergy.total\s*=\s*([\d.]+)\s+([\d.]+)\s+([\d.]+)\s+([\d.]+)"
    main_memory_energy_pattern = r"mainMemoryControllerEnergy.total\s*=\s*([\d.]+)\s+([\d.]+)\s+([\d.]+)\s+([\d.]+)"
    coherence_energy_pattern = r"coherenceEnergy.total\s*=\s*([\d.]+)\s+([\d.]+)\s+([\d.]+)\s+([\d.]+)"
    total_energy_pattern = r"TotalEnergy\s*=\s*([\d.]+)\s+([\d.]+)\s+([\d.]+)\s+([\d.]+)"

    # Function to extract a metric using the given pattern
    def extract_metric(pattern, data):
        match = re.search(pattern, data)
        if match:
            return match.group(1)
        return None

    # Extract metrics from the configuration data
    l1d_hit_rate = extract_metric(l1d_hit_rate_pattern, config_data)
    l1i_hit_rate = extract_metric(l1i_hit_rate_pattern, config_data)
    l2_hit_rate = extract_metric(l2_hit_rate_pattern, config_data)
    l3_hit_rate = extract_metric(l3_hit_rate_pattern, config_data)
    l1i_tlb_hit_rate = extract_metric(l1i_tlb_hit_rate_pattern, config_data)
    l1d_tlb_hit_rate = extract_metric(l1d_tlb_hit_rate_pattern, config_data)
    ipc = extract_metric(ipc_pattern, config_data)
    branch_prediction_accuracy = extract_metric(branch_prediction_accuracy_pattern, config_data)
    time_taken = extract_metric(total_execution_time_pattern, config_data)
    micro_op_cache_hit_rate = extract_metric(micro_op_cache_hit_rate_pattern, config_data)
    target_predictor_accuracy = extract_metric(target_predictor_accuracy_pattern, config_data)
    predicate_predictor_accuracy = extract_metric(predicate_predictor_accuracy_pattern, config_data)
    core_energy = extract_metric(core_energy_pattern, config_data)
    print(core_energy)
    # shared_cache_energy = tuple(map(float, extract_metric(shared_cache_energy_pattern, config_data).split()))
    # main_memory_energy = tuple(map(float, extract_metric(main_memory_energy_pattern, config_data).split()))
    # coherence_energy = tuple(map(float, extract_metric(coherence_energy_pattern, config_data).split()))
    # total_energy = tuple(map(float, extract_metric(total_energy_pattern, config_data).split()))

    # Print the extracted metrics
    data = {
        "L1d Hit rate": l1d_hit_rate,
        "L1i Hit rate": l1i_hit_rate,
        "L2 Hit rate": l2_hit_rate,
        "L3 Hit rate": l3_hit_rate,
        "L1i TLB hit rate": l1i_tlb_hit_rate,
        "L1d TLB hit rate": l1d_tlb_hit_rate,
        "IPC": ipc,
        "Branch Prediction Accuracy": branch_prediction_accuracy,
        "Time Taken": time_taken,
        "Micro-op Cache Hit Rate": micro_op_cache_hit_rate,
        "Target Predictor Accuracy": target_predictor_accuracy,
        "Predicate Predictor Accuracy": predicate_predictor_accuracy,
        # "Core Energy": core_energy,
        # "Shared Cache Energy": shared_cache_energy,
        # "Main Memory Controller Energy": main_memory_energy,
        # "Coherence Energy": coherence_energy,
        # "Total Energy": total_energy,
    }
    return data

extract_stats('gcc.stat')

None


{'L1d Hit rate': '0.9544035',
 'L1i Hit rate': '0.79096586',
 'L2 Hit rate': '0.9847691',
 'L3 Hit rate': '0.20170666',
 'L1i TLB hit rate': '0.9988',
 'L1d TLB hit rate': '0.9954',
 'IPC': '1.7945',
 'Branch Prediction Accuracy': '89.7722',
 'Time Taken': '2228 ',
 'Micro-op Cache Hit Rate': '0.8279',
 'Target Predictor Accuracy': '82.4859',
 'Predicate Predictor Accuracy': '97.4274'}

# Others

In [ ]:
jar_file = "/home/shashankp/Desktop/GitHub_S/Advanced-CompArch/cs810_resources/Tejas/jars/tejas.jar"
config_path = "./config_tigerLake.xml"
benchmarks = ['gcc', 'lbm', 'mcf', 'namd', 'xalancbmk']
specifications = {
    "Predictor_Mode": ['AlwaysNotTaken', 'PerfectPredictor', 'BranchPredictor', 'PAgPredictor', 'GApPredictor', 'TournamentPredictor', 'NoPredictor', 'BTB', 'GShare', 'AlwaysTaken', 'TAGE-SC-L', 'GAgpredictor', 'PApPredictor', 'BimodalPredictor', 'TAGE'],
    "MainMemoryLatency": [10, 50, 100, 500],
    "CoreFrequency": [100, 500, 2000, 5000],
    "BHRsize": [2, 4, 8, 16, 32, 64],
    "MainMemoryFrequency": [100, 500, 2000, 5000],
    "ITLB-Size": [10, 50, 200, 500], 
    "DTLB-Size": [10, 50, 200, 500],
    "NumberOfMicroOps": [500, 1000, 2000, 5000],
    "IntRegFileSize": [50, 100, 200, 400],
    "VectorRegFileSize": [50, 100, 200, 400],
    "IntDiv-Latency": [1, 10, 20],
    "IntVectorMul-Latency": [1, 5, 10, 20],
    "FloatMul-Latency": [1, 5, 10, 20],
    "FloatDiv-Latency": [1, 5, 10, 20],    
}

# Modify Config File

In [9]:
def modify_config_file(old_file, new_file, key, value):
    import xml.etree.ElementTree as ET

    def change_element_recursive(root, element_name):
        # Check if the current element's tag matches the desired element name
        if root.tag == element_name:
            return root

        # Recursively search through child elements
        for child in root:
            found_element = change_element_recursive(child, element_name)
            if found_element is not None:
                return found_element

        # Element not found in this branch
        return None

    # Parse the XML file
    tree = ET.parse(old_file)
    root = tree.getroot()

    # Call the recursive function to find the element
    v = change_element_recursive(root, key,value)
    v.text=value
    if v is None:
        print("Key doesn't exist")

    tree.write(new_file)

# template : modify_config_file('config_tigerLake.xml','ht.xml','CommunicationType','bald')

In [4]:
def modify_config_file_subtree(old_file, new_file, key1, key2, value):
    import xml.etree.ElementTree as ET

    def change_element_recursive(root, element_name):
        # Check if the current element's tag matches the desired element name
        if root.tag == element_name:
            return root

        # Recursively search through child elements
        for child in root:
            found_element = change_element_recursive(child, element_name)
            if found_element is not None:
                return found_element

        # Element not found in this branch
        return None

    # Parse the XML file
    tree = ET.parse(old_file)
    root = tree.getroot()

    # Call the recursive function to find the element
    v = change_element_recursive(root, key1)
    v1 = change_element_recursive(v, key2)
    v1.text=value
    if v is None:
        print("Key doesn't exist")

    tree.write(new_file)

modify_config_file_subtree('config_tigerLake.xml','ht1.xml','ITLB','Size','2')

# Tejas Runner

In [49]:
def run_tejas(benchmark, jar_file, config_file, stat_file):
    bench_mark_path = f"/home/shashankp/Desktop/GitHub_S/Advanced-CompArch/cs810_resources/CPU2017_benchmarks/tejas_traces/{benchmark}"
    command = (f'java -jar {jar_file} {config_file} {stat_file} {bench_mark_path}').split()
    subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Experiments

In [50]:
total = 0
for benchmark in benchmarks:
    for key in specifications:
        total += len(specifications[key])
pb = tqdm(total=total)

full_result = dict()
for benchmark in benchmarks:
    full_result[benchmark] = {}
    for key in specifications:
        full_result[benchmark][key] = {}
        for value in specifications[key]:
            pb.update(1)
            new_config = f"./config/{benchmark}_{key}_{value}.xml"
            modify_config_file(config_path, new_config, key, value)
            new_stat_file = f"./stats/{benchmark}_{key}_{value}.stat"
            run_tejas(benchmark, jar_file, new_config, new_stat_file)
            extracted_data = extract_stats(new_stat_file)
# run_tejas('gcc', jar_file, config_path, './stats/gcc.stat')

In [57]:
import os
files = os.listdir('/home/shashankp/Desktop/GitHub_S/Advanced-CompArch/cs810_resources/Tejas/src/simulator/pipeline/branchpredictor')
files = [file.split('.')[0] for file in files]
files

['AlwaysNotTaken',
 'PerfectPredictor',
 'BranchPredictor',
 'PAgPredictor',
 'GApPredictor',
 'TournamentPredictor',
 'NoPredictor',
 'BTB',
 'GShare',
 'AlwaysTaken',
 'TAGESCL',
 'GAgpredictor',
 'PApPredictor',
 'BimodalPredictor',
 'TAGE']